# Preparamos variables y funciones para la obtencion de datos

## Cargamos las variables de entorno necesarias (API KEYS)

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## Creamos funciones necesarias (JsonFormatter, FetchData, MatchCategories, etc)

### Creamos la funcion Regex para poder buscar palabras clave en las respuestas de las APIS

In [2]:
import re

# Las categorias a buscar
CATEGORIES = ["memes", "game", "gaming", "ai", "real world assets"]

# Crear una expresión regular que coincida con cualquier palabra clave
regex_pattern = r"\b(?:{})\b".format("|".join(CATEGORIES))
regex = re.compile(regex_pattern, re.IGNORECASE)

### Creamos la funcion pretty_print para poder imprimir los datos de una forma mas legible


In [3]:
import json

def pretty_print(data):
    print(json.dumps(data, indent=2))

### Creamos la funcion y variables para poder realizar las peticiones a la APIS

In [4]:
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from enum import Enum


# Enum para las APIs
class APIS(Enum):
    COINMARKET = {
        "url": "https://pro-api.coinmarketcap.com/v1/",
        "headers": {
            "Accepts": "application/json",
            "X-CMC_PRO_API_KEY": os.environ.get("API_KEY_COINMARKET"),
        },
    }
    MESSARI = {
        "url": "https://api.messari.io/",
        "headers": {
            "accept": "application/json",
            "x-messari-api-key": os.environ.get("API_KEY_MESSARI"),
        },
    }

In [5]:
# Función para obtener datos de una API
# api = Usar Enum APIS
# uri = uri de la API
# parameters = parámetros de la API
def fetch_data(api: APIS, uri: str, parameters: dict) -> dict:
    api_ = api.value
    session = Session()
    session.headers.update(api_["headers"])
    try:
        response = session.get(api_["url"] + uri, params=parameters)
        return json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        return {"error": e}

### Crear funcion para guardar archivos

In [6]:
from datetime import date

# Función para guardar los datos en un archivo JSON
def save_json(FILE: dict | list[dict], dir: str):
    dirs = dir.split("/")[:-1]
    if dirs:
        # Crear un directorio para guardar los metadatos
        os.makedirs("/".join(dirs), exist_ok=True)

    # Guardamos las categorias filtradas
    with open(dir, "w") as file:
        json.dump(FILE, file)

# Using CoinMarketAPI


## Usamos la documentacion para obtener las categorias de Coin Market API


In [7]:
# Obtenemos la lista de criptomonedas
categories_coinmarket = fetch_data(
    api=APIS.COINMARKET, uri="cryptocurrency/categories", parameters={}
)
# Obtenemos la data de las categorias
categories_coinmarket = categories_coinmarket["data"]

In [8]:
# Filtrar las categorías utilizando la expresión regular
categories_coinmarket = list(
    filter(lambda categoria: regex.search(
        categoria["title"].lower()), categories_coinmarket)
)

In [9]:
save_json(categories_coinmarket, "metadata/categories_coinmarket.json")

## Ahora que tenemos los `id` de las categorias que necesitamos (IA, Games, RWA, etc), obtenemos la data de las criptomonedas

In [18]:
cryptos_coinmarket = {}
for category in categories_coinmarket:
    cryptos = fetch_data(
        api=APIS.COINMARKET,
        uri="cryptocurrency/category",
        parameters={"id": category["id"]},
    )
    cryptos_coinmarket[category["title"]] = {"coins": cryptos["data"]["coins"]}

In [163]:
save_json(cryptos_coinmarket, f"data/json/cryptos_coinmarket_{date.today()}.json")

### Ahora pasaremos todo a `.csv` usando `pandas`

In [ ]:
import pandas as pd

df_coinmarket = pd.DataFrame()
for category in cryptos_coinmarket:
    coins: list[dict] = cryptos_coinmarket[category]["coins"]
    # dict to pandas
    df = pd.json_normalize(coins)
    df["category"] = category
    df_coinmarket = pd.concat([df_coinmarket, df])

In [160]:
df_coinmarket.head(5)

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,category,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address
0,29920,SLERF,SLERF,slerf,85,2024-03-18T09:28:39.000Z,"[memes, solana-ecosystem, presale-memes]",4.999978e+08,4.999978e+08,4.999978e+08,...,0.0000,1.994408e+08,NaN,2024-04-14T23:33:00.000Z,Presale Memes,NaN,NaN,NaN,NaN,NaN
1,30109,Pundu,PUNDU,pundu,20,2024-03-27T04:30:58.000Z,"[memes, presale-memes]",NaN,0.000000e+00,9.999680e+08,...,0.0000,1.565225e+07,NaN,2024-04-14T23:33:00.000Z,Presale Memes,NaN,NaN,NaN,NaN,NaN
0,5805,Avalanche,AVAX,avalanche,708,2020-07-13T00:00:00.000Z,"[defi, smart-contracts, three-arrows-capital-p...",7.157487e+08,3.779116e+08,4.366012e+08,...,0.5976,2.701379e+10,NaN,2024-04-14T23:33:00.000Z,Real World Assets,NaN,NaN,NaN,NaN,NaN
1,1975,Chainlink,LINK,chainlink,1726,2017-09-20T00:00:00.000Z,"[platform, defi, oracles, smart-contracts, sub...",1.000000e+09,5.871000e+08,1.000000e+09,...,0.3497,1.421805e+10,NaN,2024-04-14T23:33:00.000Z,Real World Assets,1027.0,Ethereum,ETH,ethereum,0x514910771af9ca656af840dff83e8264ecf986ca
2,8916,Internet Computer,ICP,internet-computer,231,2021-03-23T00:00:00.000Z,"[platform, distributed-computing, collectibles...",NaN,4.626845e+08,5.177064e+08,...,0.2519,6.728105e+09,NaN,2024-04-14T23:33:00.000Z,Real World Assets,8916.0,ICP,ICP,internet-computer,ryjl3-tyaaa-aaaaa-aaaba-cai


In [164]:
# save csv with the actual time
df_coinmarket.to_csv(f"data/csv/df_coinmarket_{date.today()}.csv")

# Using Messari


## Usamos la documentacion para obtener las categorias de Messari API